In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import joblib

In [7]:
data = pd.read_csv('data.csv')

data = data[data['desired_saving'] >= 5000]

X = data[['MonthlyPayment',	'IncomeTotal']]
y = data['desired_saving']

In [14]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import numpy as np

# Assuming X and y are already defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the decision tree model
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Calculate performance metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Perform cross-validation
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')

# Print performance metrics
print("Model Performance Metrics:")
print(f"R-squared (R²) Score: {r2:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Cross-validation R² scores: {cv_scores}")
print(f"Mean CV R² score: {np.mean(cv_scores):.4f}")

# Function to predict desired savings and calculate months to reach savings goal
def predict_savings_and_time(monthly_payment, total_income, savings_goal):
    input_scaled = scaler.transform([[monthly_payment, total_income]])
    predicted_savings = model.predict(input_scaled)[0]
    
    # Calculate disposable income
    disposable_income = total_income - monthly_payment
    
    # Cap the predicted savings at 50% of disposable income
    max_savings = disposable_income * 0.5
    predicted_savings = min(predicted_savings, max_savings)
    
    savings_percentage = (predicted_savings / total_income) * 100
    months_to_goal = max(min(int(np.ceil(savings_goal / predicted_savings)), 96), 1) if predicted_savings > 0 else 96
    
    return predicted_savings, savings_percentage, months_to_goal

# Example usage
monthly_payment = 3000
total_income = 7000
savings_goal = 20000

predicted_savings, savings_percentage, months_to_goal = predict_savings_and_time(monthly_payment, total_income, savings_goal)

print(f"\nFor monthly payment of ${monthly_payment} and total income of ${total_income}:")
print(f"Predicted desired savings: ${predicted_savings:.2f}")
print(f"Savings percentage of total income: {savings_percentage:.2f}%")
print(f"Months to reach savings goal of ${savings_goal}: {months_to_goal}")

# Print feature importances
feature_importance = model.feature_importances_
features = X.columns
for f, imp in zip(features, feature_importance):
    print(f"Importance of {f}: {imp:.4f}")

Model Performance Metrics:
R-squared (R²) Score: -0.0359
Mean Squared Error (MSE): 4692850.4549
Root Mean Squared Error (RMSE): 2166.2988
Mean Absolute Error (MAE): 1355.1639
Cross-validation R² scores: [-0.04474391  0.00770018 -0.02697778  0.01467063 -0.08212497]
Mean CV R² score: -0.0263

For monthly payment of $3000 and total income of $7000:
Predicted desired savings: $2000.00
Savings percentage of total income: 28.57%
Months to reach savings goal of $20000: 10
Importance of MonthlyPayment: 0.7216
Importance of IncomeTotal: 0.2784


C:\Users\Fatima\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [16]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
import pickle
import numpy as np
import pandas as pd

# Load your data
# Replace this with your actual data loading code
data = pd.read_csv('data.csv')
data = data[data['desired_saving'] >= 5000]

X = data[['MonthlyPayment',	'IncomeTotal']]
y = data['desired_saving']

# Create and fit the scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create and fit the model
model = DecisionTreeRegressor(random_state=42)
model.fit(X_scaled, y)

# Save the scaler and model using pickle
with open('savings_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('savings_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Scaler and model saved successfully using pickle.")

# Function to predict desired savings and calculate months to reach savings goal
def predict_savings_and_time(scaler, model, monthly_payment, total_income, savings_goal):
    input_scaled = scaler.transform([[monthly_payment, total_income]])
    predicted_savings = model.predict(input_scaled)[0]
    
    # Calculate disposable income
    disposable_income = total_income - monthly_payment
    
    # Cap the predicted savings at 50% of disposable income
    max_savings = disposable_income * 0.5
    predicted_savings = min(predicted_savings, max_savings)
    
    savings_percentage = (predicted_savings / total_income) * 100
    months_to_goal = max(min(int(np.ceil(savings_goal / predicted_savings)), 96), 1) if predicted_savings > 0 else 96
    
    return predicted_savings, savings_percentage, months_to_goal

# Test the prediction function
with open('savings_scaler.pkl', 'rb') as f:
    test_scaler = pickle.load(f)

with open('savings_model.pkl', 'rb') as f:
    test_model = pickle.load(f)

monthly_payment = 3000
total_income = 7000
savings_goal = 30000

predicted_savings, savings_percentage, months_to_goal = predict_savings_and_time(
    test_scaler, test_model, monthly_payment, total_income, savings_goal
)

print(f"\nTest Prediction:")
print(f"For monthly payment of ${monthly_payment} and total income of ${total_income}:")
print(f"Predicted desired savings: ${predicted_savings:.2f}")
print(f"Savings percentage of total income: {savings_percentage:.2f}%")
print(f"Months to reach savings goal of ${savings_goal}: {months_to_goal}")

Scaler and model saved successfully using pickle.

Test Prediction:
For monthly payment of $3000 and total income of $7000:
Predicted desired savings: $2000.00
Savings percentage of total income: 28.57%
Months to reach savings goal of $30000: 15


C:\Users\Fatima\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
